# Create convolutional neural network model

## Imports

In [6]:
#import numpy as np
#import h5py
from tflearn import DNN
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_augmentation import ImageAugmentation
from tflearn.data_utils import image_preloader
from sklearn.model_selection import train_test_split

## Get dataset

In [9]:
# Load path/class_id image file:
data_folder = 'images/final_dataset'

# Build the preloader array, resize images to 640x480
X, Y = image_preloader(data_folder, image_shape=(480, 640), mode='folder', categorical_labels=True, normalize=True, files_extension='.jpg')

### Image augmentation
We use image augmentation to create a better training effect for real world situations. We randomly blur images and rotate them.

In [4]:
# Real-time data augmentation (This is only used while training the DNN)
img_aug = ImageAugmentation()
# Add blur so even when our vehicle is moving it can detect images
img_aug.add_random_blur(0.5)
# Add a random rotation of 20 degrees to even capture rotated images (For our project max. 10 but your never know)
img_aug.add_random_rotation(max_angle=20.0)

### Convolutional neural network model

In [7]:
# Convolutional network building
network = input_data(shape=[None, 620, 480, 3],
                     data_augmentation=img_aug)
network = conv_2d(network, 16, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 16, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 32, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 7, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

model = DNN(network, tensorboard_verbose=1)

### Load the dataset 

In [ ]:
# Load the final dataset
#h5f = h5py.File('data/final_dataset.h5', 'r')
#X = h5f['X']
#Y = h5f['Y']

# Cut Dataset in training and test set
X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

### Train the model and save it

In [ ]:
# Start the trainig process
model.fit(X, Y, n_epoch=5, shuffle=True, validation_set=0.2, show_metric=True, batch_size=128, validation_batch_size=128, run_id='final_model')

# Evaluate performance on test set
model.evaluate(X_test, Y_test)

# Save the model
model.save('models/final_model')

# Close the database connection
h5f.close()